# Homework 3
Due 9/20 @ 11pm:

Create a longitudinal model of a tilt rotor aircraft. You should use the model on pg. 182 
of Stevens and Lewis and add the tilt rotor dynamic by creating a 5th input, to 
represent the tilt of the motors.

Trim the aircraft at 500 ft/s and at 100 ft/s and design a pitch rate controller for both 
trim conditions.

What challenges do you encounter with regard to the aerodynamics at 
the 100 ft/s trim condition?

In [1]:
import sys
sys.path.insert(0, '../python/casadi_f16')
import f16
import control
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

from analysis import loop_analysis, rlocus, bode

## Function Definitions

In [5]:
# Trimming function for a tilt rotor aircraft
def trim_tra():
    return()

In [6]:
# Atmosphere function
def ADC(VT, ALT):
    R0 = 2.377E-3                # Sea level density
    TFAC = 1.0 - 0.703E-5*ALT
    T = 519.0 + TFAC             # Temperature
    if(ALT > 35000.0): T = 390.0
    RHO = R0 * TFAC**4.14        # Density
    GAM = 1.4
    R = 1716.3
    
    AMACH = VT/sqrt(GAM*R*T)
    QBAR = 0.5*RHO*VT**2
    return(AMACH, QBAR)

# Dynamics of a transport aircraft, returning dx/dt
def transp(time, x, u):

    # Definitions
    S = 2170      # Area of wing
    CBAR = 17.5   # Average chord
    MASS = 5.0E3  # Mass
    IYY = 4.1E6   # Moment of Inertia about y axis
    TSTAT = 6.0E4 # Static thrust
    DTDV = -38.0  # Change in thrust w.r.t. velocity
    ZE = 2.0      # 
    CDCLS = 0.042 #

    # Per degree
    CLA = 0.085   # Coefficient of lift
    CMA = -0.022  # Coefficient of moment
    CMDE = -0.016 # Coefficient of moment

    # Per radian
    CMQ = -16.0   # 
    CMADOT = -6.0 # Change in coefficient of moment
    CLADOT = 0.0  # Change in coefficient of lift

    RTOD = 57.29578 # Radians to degrees conversion
    GD = 32.17      # 

    # Assigning input vector
    THTL = u[0]
    ELEV = u[1]
    XCG = u[2]
    LAND = u[3]

    # Assigning state vector
    VT = x[0]
    ALPHA = RTOD * x[1]
    THETA = x[2]
    Q = x[3]
    H = x[4]

    [MACH, QBAR] = ADC(VT, H)
    QS = QBAR*S
    SALP = sin(x[1])
    CALP = cos(x[1])
    GAM = THETA - x[1]
    SGAM = sin(GAM)
    CGAM = cos(GAM)

    # Correcting for landing
    coeff = ca.if_else(LAND == 0,
      [0.20, 0.016, 0.05, 0.0, 0.0],
      ca.if_else(LAND == 1,
                [1.0, 0.08, -0.20, 0.02, -0.05],
                99))
    if(coeff[0] == 99):
        print('Landing Gear & Flaps?')
    else:
        CLO = coeff[0]
        CDO = coeff[1]
        CMO = coeff[2]
        DCDG = coeff[3]
        DCMG = coeff[4]

    THR = (TSTAT + VT*DTDV) * max(THTL, 0) # Thrust
    CL = CLO + CLA*ALPHA                   # Lift Coefficient
    CD = DCDG + CDO + CDCLS*CL**2          # Drag Coefficient

    # Moment Coefficient
    CM = DCMG + CMO + CMA*ALPHA + CMDE + ELEV + CL*(XCG - 0.25)

    # Time derivative of state equation
    xd[0] = (THR*CALP - QS*CD)/MASS - GD*SGAM
    xd[1] = (-THR*SALP - QS*CL + MASS*(VT*Q + GD*CGAM))/(MASS*VT + QS*CLADOT)
    xd[2] = Q
    xd[3] = (QS*CBAR*(CM + D) + THR*ZE)/IYY
    xd[4] = VT*SGAM
    xd[5] = VT*CGAM
    return xd

## Controller

In [ ]:
# Pitch Rate Controller
P = 2.5
I = 0.5
H_PI = P + I * (1/ s)
print('H_PI = ',H_PI)

G0 = H_PI*sys3[1, 0]
print('G0 = ',G0)

Gc = control.feedback(G0, 1)

t, y = control.step_response(Gc, T=np.linspace(0, 1, 1000))
plt.figure(figsize=(10, 5))
plt.plot(t, 1*y)
plt.grid()
plt.show()

## Analysis